<a href="https://colab.research.google.com/github/david-dirring/data-wrangler-in-ml-world/blob/master/_notebooks/2021_06_04_no_more_just_hoping_my_data_is_right.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Is your batch prediction data is similar enough to your train data? 
> Below you will learn how to quickly build a simple process to check whether there are any data issues by building a simple model to predict whether a random record is part of your "Train" or "Inference" dataset.

[](my_icons/No_More_Just_Hoping.png)


I have a project at work where we **batch load predictions** into an app daily. 

We have a training process that we rerun weekly, but our batch prediction process (test-inference process) runs multiple times per day (we may not run multiple times per day forever, still in pilot stage).  Much of the data changes intraday, **so I have to be sure that the inference data that we feed into our weekly trained model has the same charateristics as the data that it trained on.**

[](my_icons/no_more_just_hoping_data_is_right_training.png)
[](my_icons/no_more_just_hoping_data_is_right_inference.png)
